# LLM Generation Parameters

In [1]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

## Import Libraries

In [8]:
import os
import json
from dotenv import load_dotenv
from openai import OpenAI
from anthropic import Anthropic
from IPython.display import Markdown, display

In [9]:
# Always remember to do this! - What does this do?
load_dotenv(override=True)

True

## Exploring Parameters

### OpenAI Chat Completions API (most widely used)

`OpenAI` Official documentation is here - https://platform.openai.com/docs/api-reference/chat/create Refer to this for a full set of parameters that we'll later explore. 

The most used parameters are as listed below, with definitions from official `OpenAI` API documentation. These are considered "common" because they fundamentally control how an autoregressive language model samples tokens to generate output. 

- **max_tokens** (only *max_completion_tokens* is supported by o1 series; no default): The maximum number of completion tokens that may be used over the course of the run. The run will make a best effort to use only the number of completion tokens specified, across multiple turns of the run. If the run exceeds the number of completion tokens specified, the run will end with status `incomplete`.
- **temperature** (*1 by default*): What sampling temperature to use, between 0 and 2. Higher values like 0.8 will make the output more random, while lower values like 0.2 will make it more focused and deterministic. We generally recommend altering this or top_p but not both.
- **top_p** (*1 by default*): An alternative to sampling with temperature, called nucleus sampling, where the model considers the results of the tokens with top_p probability mass. So 0.1 means only the tokens comprising the top 10% probability mass are considered.
We generally recommend altering this or temperature but not both.
- **top_k**: not available in OpenAI client library
- **frequency_penalty** (*0 by default*): Number between -2.0 and 2.0. Positive values penalize new tokens based on their existing frequency in the text so far, decreasing the model's likelihood to repeat the same line verbatim.
- **presence_penalty** (*0 by default*): Number between -2.0 and 2.0. Positive values penalize new tokens based on whether they appear in the text so far, increasing the model's likelihood to talk about new topics.
- **stop** (*null by default*): Not supported with latest reasoning models o3 and o4-mini. Up to 4 sequences where the API will stop generating further tokens. The returned text will not contain the stop sequence.

### Anthropic's Messages API

It's good to note that Anthropic's `Messages` API contains similar parameters except a few differences:

- **max_tokens** is used as is
- **temperature** has a value between 0 and 1
- **top_p** is as is
- **top_k** is also available: Only sample from the top K options for each subsequent token. Used to remove "long tail" low probability responses. Recommended for advanced use cases only. You usually only need to use temperature.
- **frequency_penalty** and **presence_penalty** parameters are not available.
- **stop_sequences** is used instead of **stop**

### Some important considerations

- Exact naming conventions of parameters can vary, but functionality is same (like we saw with Anthropic's API).
- The optimal range or impact of a parameter might differ between models, even if general principle is same. A temperature of 1.0 might be very wild for one model but only moderately creative for another.
- Differences between different parameter configurations will be more prominent in larger LLMs, as with smaller LLMs, the "spread" of probabilities for the next token might not be as wide.

### Setting temperature vs. top_p for different use cases: 

| Use Case               | Temperature | Top_p | Description                                                                                        |
| :--------------------- | :---------- | :---- | :------------------------------------------------------------------------------------------------- |
| Code Generation        | 0.2         | 0.1   | Generates code that adheres to established patterns and conventions. Output is more deterministic and focused. Useful for generating syntactically correct code. |
| Creative Writing       | 0.7         | 0.8   | Generates creative and diverse text for storytelling. Output is more exploratory and less constrained by patterns. |
| Chatbot Responses      | 0.5         | 0.5   | Generates conversational responses that balance coherence and diversity. Output is more natural and engaging. |
| Code Comment Generation | 0.3         | 0.2   | Generates code comments that are more likely to be concise and relevant. Output is more deterministic and adheres to conventions. |
| Data Analysis Scripting | 0.2         | 0.1   | Generates data analysis scripts that are more likely to be correct and efficient. Output is more deterministic and focused. |
| Exploratory Code Writing | 0.6         | 0.7   | Generates code that explores alternative solutions and creative approaches. Output is less constrained by established patterns. |

References from here: https://community.openai.com/t/cheat-sheet-mastering-temperature-and-top-p-in-chatgpt-api/172683/10

### Setting Up

In [10]:
# Use either Llama3.2 (3B) with Ollama
!ollama pull llama3.2    

# Initialize the OpenAI client to connect to Ollama
llm_api = OpenAI(base_url='http://localhost:11434/v1', 
                 api_key='ollama') # note that llm_api is a good variable name, because the LLM is indeed exposed locally as an API by Ollama!

# Define the model to use
model_name = "llama3.2" # Ensure this matches the pulled model in Ollama

pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠏ pulling manifest ⠋ pulling manifest ⠙ pulling manifest 
pulling dde5aa3fc5ff: 100% ▕██████████████████▏ 2.0 GB                         
pulling 966de95ca8a6: 100% ▕██████████████████▏ 1.4 KB                         
pulling fcc5a6bec9da: 100% ▕██████████████████▏ 7.7 KB                         
pulling a70ff7e570d9: 100% ▕██████████████████▏ 6.0 KB                         
pulling 56bb8bd477a5: 100% ▕██████████████████▏   96 B                         
pulling 34bb5ab01051: 100% ▕██████████████████▏  561 B                         
verifying sha256 digest 
writing manifest 
success 


In [11]:
# # Or use DeepSeek-R1-Distill-Llama-70B from Groq
# llm_api = OpenAI(api_key=groq_api_key, base_url="https://api.groq.com/openai/v1")

# # Define the model to use
# model_name = "deepseek-r1-distill-llama-70b"

In [12]:
#Helper function for text generation
def generate_and_display(prompt, param_name, param_value, num_runs=1, **kwargs):
    """
    Helper function to make a call to the LLM and display results.
    Runs multiple times for stochastic parameters to highlight differences.
    """
    print(f"\n--- Demonstrating {param_name}={param_value} ---")
    print(f"Prompt: \"{prompt}\"")
    print(f"Other consistent parameters: {kwargs}")

    messages = [
        {"role": "system", "content": "You are a helpful and concise AI assistant for a business."},
        {"role": "user", "content": prompt},
    ]

    for i in range(num_runs):
        print(f"\n--- Run {i+1} ---")
        try:
            response = llm_api.chat.completions.create(
                model=model_name,
                messages=messages,
                **{param_name: param_value, **kwargs}
            )
            print("Generated Text:")
            answer = response.choices[0].message.content.strip()
            display(Markdown(answer)) # .strip() removes leading/trailing whitespace for cleaner output
        except Exception as e:
            print(f"An error occurred: {e}")
            print("Ensure Ollama is running and the model is pulled. Skipping further runs for this parameter setting.")
            break # Exit loop if an error occurs

    print("\n--- End Demonstration for this parameter setting ---\n" + "="*80)

### 1. Temperature

**Commercial Use Cases**: Content generation (marketing copy, ad variations, blog post ideas), creative brainstorming for product features, unique customer engagement messages.

**What to expect:**
- **Low Temperature**: Look for very similar, factual, and direct outputs across runs. Less "fluff," more to the point.
    - E.g., Consistent, factual product bullet points for a catalog
- **Medium Temperature**: Outputs will start showing more natural language, some variation in phrasing, but still coherent.
    - E.g., Engaging marketing copy for a landing page
- **High Temperature**: Expect more varied vocabulary, potentially more metaphorical or unusual phrasing, and noticeable differences between runs. It might sometimes deviate slightly from the core prompt if pushed too high.
    - E.g., Brainstorming unique ad slogans or creative concepts for a campaign

In [7]:
common_prompt_temp = """
Reliance is heavily investing in Generative AI to serve India's 'next billion users'. 
Envision a new GenAI-powered service that leverages Reliance's massive user base and diverse data. 
First think about which sector you want to pick for this GenAI-powered service, and it can be one of: Oil to Chemicals, Digital Services, 
Retail, New Energy & Materials, Media & Entertainment, Oil & Gas Exploration & Production, Financial Services, Textiles, Infrastructure.
Then, describe this service, focusing on its innovative features and how it will enhance daily life for a typical Indian user.
Keep the answer less than 70 tokens."""

In [8]:
# Example 1.1: Low Temperature
generate_and_display(
    common_prompt_temp,
    "temperature", 0.1,
    num_runs=3, # Run multiple times to show consistency
    max_completion_tokens=70
)


--- Demonstrating temperature=0.1 ---
Prompt: "
Reliance is heavily investing in Generative AI to serve India's 'next billion users'. 
Envision a new GenAI-powered service that leverages Reliance's massive user base and diverse data. 
First think about which sector you want to pick for this GenAI-powered service, and it can be one of: Oil to Chemicals, Digital Services, 
Retail, New Energy & Materials, Media & Entertainment, Oil & Gas Exploration & Production, Financial Services, Textiles, Infrastructure.
Then, describe this service, focusing on its innovative features and how it will enhance daily life for a typical Indian user.
Keep the answer less than 70 tokens."
Other consistent parameters: {'max_completion_tokens': 70}

--- Run 1 ---
Generated Text:


I propose a GenAI-powered service in the Retail sector: "Jagat" - a personalized e-commerce platform.

Jagat leverages Reliance's diverse data to offer AI-driven product recommendations, inventory management, and supply chain optimization. 

Key features:

* AI-powered shopping assistant: provides users with tailored suggestions based on their purchase history and preferences.
* Virtual try-on: uses AR/VR technology for seamless virtual product demonstrations.
* Smart home delivery: integrates with customers' smart home devices for efficient package tracking and delivery.

Jagat aims to enhance daily life by making online shopping more convenient, personalized, and enjoyable for Indian users.


--- Run 2 ---
Generated Text:


I propose a GenAI-powered service in the Retail sector: "Jagat" - an AI-driven personalized shopping assistant.

Jagat uses machine learning to analyze users' purchase history, preferences, and lifestyle to offer tailored product recommendations, exclusive deals, and personalized services. The service integrates with Reliance's vast network of retail stores, e-commerce platforms, and logistics.

Innovative features include:

* AI-powered styling suggestions for clothing and home decor
* Personalized meal planning and grocery delivery
* Virtual try-on for fashion items
* Real-time price tracking and alerts

Jagat enhances daily life by making shopping more convenient, efficient, and enjoyable. By leveraging Reliance's massive user base and diverse data, Jagat aims to become the go-to retail assistant for India's 'next billion users'.


--- Run 3 ---
Generated Text:


I propose a GenAI-powered service in the Retail sector: "Jagat" - a personalized shopping assistant.

Jagat leverages Reliance's diverse data to offer users:

* AI-driven product recommendations based on purchase history and preferences
* Virtual try-on for clothing and accessories
* Augmented reality (AR) shopping experiences
* Personalized customer service using chatbots and voice assistants

Jagat aims to enhance daily life by making shopping more convenient, efficient, and enjoyable. By harnessing the power of GenAI, Jagat can provide users with a seamless and personalized shopping experience, revolutionizing the way Indians shop and interact with retail services.


--- End Demonstration for this parameter setting ---


In [9]:
# Example 1.2: Medium Temperature
generate_and_display(
    common_prompt_temp,
    "temperature", 0.7,
    num_runs=3, # Run multiple times to show controlled variation
    max_completion_tokens=70
)


--- Demonstrating temperature=0.7 ---
Prompt: "
Reliance is heavily investing in Generative AI to serve India's 'next billion users'. 
Envision a new GenAI-powered service that leverages Reliance's massive user base and diverse data. 
First think about which sector you want to pick for this GenAI-powered service, and it can be one of: Oil to Chemicals, Digital Services, 
Retail, New Energy & Materials, Media & Entertainment, Oil & Gas Exploration & Production, Financial Services, Textiles, Infrastructure.
Then, describe this service, focusing on its innovative features and how it will enhance daily life for a typical Indian user.
Keep the answer less than 70 tokens."
Other consistent parameters: {'max_completion_tokens': 70}

--- Run 1 ---
Generated Text:


I propose a GenAI-powered service in the Retail sector: "DreamMart".

DreamMart leverages Reliance's vast data and user base to create a personalized, AI-driven shopping experience. Users receive customized recommendations based on their preferences, purchase history, and lifestyle.

Innovative features include:

* Virtual try-on for clothing and accessories
* AI-powered product reviews and ratings
* Personalized offers and discounts
* Smart inventory management for in-store experiences

By enhancing daily life with DreamMart, typical Indian users can enjoy a seamless shopping experience, discover new products, and make informed purchasing decisions.


--- Run 2 ---
Generated Text:


I propose a GenAI-powered "EcoLife" service in the Retail sector.

**EcoLife**:

* A personal shopping assistant using AI-driven recommendations, suggesting eco-friendly products based on users' preferences and daily habits.
* Integrates with Reliance's vast data repository to analyze user behavior, offer personalized discounts, and provide real-time suggestions for sustainable living.
* Features:
	+ "Green Profile": Users create a digital profile showcasing their environmental footprint, helping them track progress towards sustainability goals.
	+ "Eco-Alerts": AI-powered notifications alerting users about sales on eco-friendly products, new arrivals, or special promotions.

By leveraging Reliance's massive user base and diverse data, EcoLife aims to make sustainable living more accessible, convenient, and affordable for the Indian masses.


--- Run 3 ---
Generated Text:


I propose Reliance's GenAI-powered service in the Retail sector: "Jagruk Mart" (Aware Market).

**Service Description:** Jagruk Mart uses GenAI to create a personalized, voice-activated shopping assistant that helps users discover and purchase products based on their preferences, location, and needs.

**Innovative Features:**

* AI-driven product recommendations
* Real-time inventory updates
* Personalized offers and discounts
* Integration with e-wallets and mobile payments

**Enhancing Daily Life:** Jagruk Mart enhances daily life by streamlining shopping experiences, saving time, and providing users with relevant product suggestions. It also fosters a more inclusive retail ecosystem, catering to the diverse needs of India's "next billion users."


--- End Demonstration for this parameter setting ---


In [10]:
# Example 1.3: High Temperature
generate_and_display(
    common_prompt_temp,
    "temperature", 1.5, # Pushing temperature higher for more dramatic effect
    num_runs=3, # Run multiple times to show increased randomness
    max_completion_tokens=70
)


--- Demonstrating temperature=1.5 ---
Prompt: "
Reliance is heavily investing in Generative AI to serve India's 'next billion users'. 
Envision a new GenAI-powered service that leverages Reliance's massive user base and diverse data. 
First think about which sector you want to pick for this GenAI-powered service, and it can be one of: Oil to Chemicals, Digital Services, 
Retail, New Energy & Materials, Media & Entertainment, Oil & Gas Exploration & Production, Financial Services, Textiles, Infrastructure.
Then, describe this service, focusing on its innovative features and how it will enhance daily life for a typical Indian user.
Keep the answer less than 70 tokens."
Other consistent parameters: {'max_completion_tokens': 70}

--- Run 1 ---
Generated Text:


Sector: Digital_services

Reliance's GenAI-powered service: "SmartLife Partner"

This AI-driven platform leverages Reliance's vast data assets to provide personalized lifestyle solutions, bridging the digital divide for India's 'next billion users.' The service seamlessly integrates with smartphones to:

1. Auto-detect and assist in health monitoringsolutions using computer vision analytics.
2. Recognize local accents using voice-assist feature & respond instantly in more than 10 langauges.
3. Predictively offer travel suggestions, based on real-time traffic patterns.

Smartlife Partner uses artificial learning networks that help identify the exact need & needs and suggest options using this advanced information analysis with every passing day.

With ease & access from anywhere, 'life can get easy & smarter in a flash'.


--- Run 2 ---
Generated Text:


Sector: Digital Services

Service Name: ReliaMentor

ReliaMentor combines AI-assisted learning platforms with personalized skill development. 

- **Personalized Career Guidance**: AI-drives an extensive network assessment to provide guidance for students/learners choosing industries for themselves.
  
This service empowers daily users by helping them discover apt career choices using advanced, large dataset-driven assessments powered by ReLiannce's rich content and comprehensive expertise from over a Billion users in digitl spaces - including its popular educational programs (Jio TV & Digital Movies) ,JioMeet  and Quizzes.


--- Run 3 ---
Generated Text:


Let's focus on Education.

**GenAI-powered "DreamCareer" - A Virtual Career Counsellor**

For Reliance: Developing advanced AI technology to create an immersive, AI-assisted career guidance platform, integrating machine learning insights, mentorship, and resources for Indians aspiring to reach new goals. In-app tools offer guidance on resume building, interviewing strategy, skill development course recommendations, online learning platforms, networking opportunities and employability enhancement training! The 'DreamCareer' Gen AI service brings career goals within reach of millions, redefining India's future workforce.


--- End Demonstration for this parameter setting ---


### 2. Max_tokens

**Commercial Use Cases**: Chatbot response length control, email subject line generation, tweet generation, summarizing lengthy documents for internal reports, generating specific-length ad copy.

**What to expect**: 

- The generated output will strictly adhere to the `max_tokens` limit, regardless of whether the summary is complete.
- Depending on the use case, the `max_tokens` parameter can be set:
    - Very few tokens (e.g., for a headline or quick alert)
    - Moderate tokens (e.g., for an executive summary or internal Slack message)
    - More tokens (e.g., for a detailed summary section in a newsletter or report)

In [13]:
common_prompt_max_tokens = """
Imagine you're a data scientist working with an IPL team. 
Describe how Generative AI could be used to create novel training drills or strategic simulations for batsmen, considering the unique challenges of Indian pitches and varied bowling styles."""

In [14]:
# Example 2.1: Very few tokens
generate_and_display(
    common_prompt_max_tokens + " Complete response in less than 15 tokens.",
    "max_tokens", 15,
    temperature=0.5
)


--- Demonstrating max_tokens=15 ---
Prompt: "
Imagine you're a data scientist working with an IPL team. 
Describe how Generative AI could be used to create novel training drills or strategic simulations for batsmen, considering the unique challenges of Indian pitches and varied bowling styles. Complete response in less than 15 tokens."
Other consistent parameters: {'temperature': 0.5}

--- Run 1 ---
Generated Text:


As a data scientist, I'd leverage Generative AI to generate:

1


--- End Demonstration for this parameter setting ---


In [13]:
# Example 2.2: Moderate tokens
generate_and_display(
    common_prompt_max_tokens,
    "max_tokens", 50,
    temperature=0.5
)


--- Demonstrating max_tokens=50 ---
Prompt: "
Imagine you're a data scientist working with an IPL team. 
Describe how Generative AI could be used to create novel training drills or strategic simulations for batsmen, considering the unique challenges of Indian pitches and varied bowling styles."
Other consistent parameters: {'temperature': 0.5}

--- Run 1 ---
Generated Text:


As a data scientist working with an IPL team, I'd like to explore the potential of Generative AI in creating novel training drills and strategic simulations for batsmen. Here's how:

**Novel Training Drills:**

1. **Simulated


--- End Demonstration for this parameter setting ---


In [14]:
# Example 2.3: More tokens
generate_and_display(
    common_prompt_max_tokens,
    "max_tokens", 150,
    temperature=0.5
)


--- Demonstrating max_tokens=150 ---
Prompt: "
Imagine you're a data scientist working with an IPL team. 
Describe how Generative AI could be used to create novel training drills or strategic simulations for batsmen, considering the unique challenges of Indian pitches and varied bowling styles."
Other consistent parameters: {'temperature': 0.5}

--- Run 1 ---
Generated Text:


As a data scientist working with an IPL team, I'd love to explore the potential of Generative AI in creating innovative training drills and strategic simulations for batsmen. Here's how:

**Generative AI Applications**

1. **Batting Drills**: Utilize Generative AI to create personalized batting drills based on individual batsmen's strengths, weaknesses, and playing styles. The algorithm can generate a variety of scenarios, such as:
	* Mimicking specific bowlers' deliveries (e.g., Jofra Archer's bouncers or Jasprit Bumrah's yorkers).
	* Simulating different pitch conditions (e.g., the Gargantuan turner at Eden Gardens or the seam-friendly surfaces at Motera Stadium).


--- End Demonstration for this parameter setting ---


In [15]:
#Example 2.4: Even more max_tokens
generate_and_display(
    common_prompt_max_tokens,
    "max_tokens", 1000,
    temperature=0.5
)


--- Demonstrating max_tokens=1000 ---
Prompt: "
Imagine you're a data scientist working with an IPL team. 
Describe how Generative AI could be used to create novel training drills or strategic simulations for batsmen, considering the unique challenges of Indian pitches and varied bowling styles."
Other consistent parameters: {'temperature': 0.5}

--- Run 1 ---
Generated Text:


As a data scientist working with an IPL team, I'd love to explore the potential of Generative AI in creating novel training drills and strategic simulations for batsmen. Here's how:

**Data Collection**

To create effective generative models, we'll need access to vast amounts of high-quality data on Indian pitches, bowling styles, and batting performances. This can include:

1. Match footage and analysis from various IPL matches.
2. Pitch reports and weather conditions.
3. Bowling action data (e.g., spin, pace, trajectory).
4. Batting performance metrics (e.g., strike rate, average, boundaries).

**Generative AI Models**

We'll employ Generative Adversarial Networks (GANs) or Variational Autoencoders (VAEs) to generate novel training drills and strategic simulations for batsmen.

1. **Pitch Generator**: Train a GAN or VAE to generate new pitch scenarios based on historical data, taking into account factors like spin, pace, and weather conditions.
2. **Bowling Style Simulator**: Develop a model that can simulate different bowling styles (e.g., leg-spin, off-spin, fast bowling) with varying degrees of difficulty.
3. **Batting Performance Predictor**: Train a VAE to predict batting performance metrics (e.g., strike rate, average, boundaries) based on pitch and bowling style inputs.

**Training Drill Generator**

Using the generated pitch scenarios and bowling style simulations, we'll create novel training drills for batsmen:

1. **Adaptive Training Drills**: Generate training drills that adapt to the batsman's current form and strengths.
2. **Scenario-Based Training**: Create simulations of specific match scenarios (e.g., chasing down a target, facing pressure situations).
3. **Personalized Training Plans**: Develop customized training plans for each batsman based on their performance metrics and preferences.

**Strategic Simulation**

To aid the team's strategic decisions during matches:

1. **Real-Time Pitch Analysis**: Use GANs or VAEs to generate real-time pitch analysis, providing insights into the current conditions.
2. **Bowling Style Prediction**: Develop a model that predicts the likelihood of different bowling styles being used in the match.
3. **Batting Strategy Optimization**: Train a machine learning model to optimize batting strategies based on historical data and real-time inputs.

**Implementation**

To integrate these generative models into our training processes:

1. **Data Integration**: Integrate generated data with existing datasets to create a comprehensive knowledge graph of Indian pitches, bowling styles, and batting performances.
2. **Training Software**: Develop custom software to visualize and interact with the generative models, enabling coaches and trainers to easily explore and adapt the simulations.
3. **Real-Time Feedback Loops**: Implement real-time feedback loops to ensure that batsmen receive immediate, actionable insights from the generative models.

By leveraging Generative AI in this way, we can create novel training drills and strategic simulations that cater to the unique challenges of Indian pitches and varied bowling styles, ultimately enhancing our team's performance and competitiveness.


--- End Demonstration for this parameter setting ---


### 3. Top_p (Nucleus sampling)

**Commercial Use Cases**: Generating diverse customer service responses (without going off-topic), creating varied product names, generating slightly different content variations for A/B testing.

**What to Observe**:

- **Low** top_p: Outputs will be very similar across runs, often picking the most statistically common words/phrases. Less variation in phrasing.

- **Medium** top_p: Some variations in phrasing will emerge, but still within a coherent and expected range.

- **High** top_p: More diverse and potentially more "creative" word choices and phrasing, showing greater variation across runs.

In [16]:
common_prompt_top_p = """
A customer is asking about our new premium coffee blend, that rivals Starbucks, Peets Coffee and Cafe Coffee Day. 
Provide a welcoming response that also highlights its unique flavor notes."""

In [17]:
# Example 3.1: Low top_p (for a very standard, consistent customer service greeting)
generate_and_display(
    common_prompt_top_p,
    "top_p", 0.1,
    num_runs=3,
    temperature=0.7,
    max_tokens=80
)


--- Demonstrating top_p=0.1 ---
Prompt: "
A customer is asking about our new premium coffee blend, that rivals Starbucks, Peets Coffee and Cafe Coffee Day. 
Provide a welcoming response that also highlights its unique flavor notes."
Other consistent parameters: {'temperature': 0.7, 'max_tokens': 80}

--- Run 1 ---
Generated Text:


"Welcome to [Your Business Name]! We're thrilled you're interested in our new premium coffee blend. I'd be happy to tell you more about it.

Our signature blend is carefully crafted to rival the best of the industry, and we're confident you'll love it just as much as our loyal customers do. What sets us apart is its unique flavor profile, which combines notes of rich chocolate


--- Run 2 ---
Generated Text:


"Welcome to [Your Business Name]! We're thrilled you're interested in our new premium coffee blend. I'd be happy to tell you more about it.

Our expertly crafted blend is indeed a game-changer, and we're confident it will satisfy even the most discerning palates. With notes of rich chocolate, velvety smooth caramel, and hints of exotic spices like cardam


--- Run 3 ---
Generated Text:


"Welcome to [Your Business Name]! We're thrilled you're interested in our new premium coffee blend. I'd be happy to tell you more about it.

Our expertly crafted blend is indeed a game-changer, and we're confident it will satisfy even the most discerning palates. With notes of rich chocolate, velvety smooth caramel, and hints of exotic spices like cardam


--- End Demonstration for this parameter setting ---


In [18]:
# Example 3.2: Medium top_p (for a friendly, slightly varied customer service interaction)
generate_and_display(
    common_prompt_top_p,
    "top_p", 0.5,
    num_runs=3,
    temperature=0.7,
    max_tokens=80
)


--- Demonstrating top_p=0.5 ---
Prompt: "
A customer is asking about our new premium coffee blend, that rivals Starbucks, Peets Coffee and Cafe Coffee Day. 
Provide a welcoming response that also highlights its unique flavor notes."
Other consistent parameters: {'temperature': 0.7, 'max_tokens': 80}

--- Run 1 ---
Generated Text:


Here's a warm and inviting response to the customer's inquiry:

"Thank you for your interest in our new premium coffee blend! We're thrilled to introduce our latest creation, carefully crafted to rival the best of the industry - Starbucks, Peet's Coffee, and Cafe Coffee Day.

Our expert roasters have worked tirelessly to bring you a truly exceptional cup, with a rich and smooth flavor profile that


--- Run 2 ---
Generated Text:


Here's a warm and inviting response to the customer's inquiry:

"Thank you for your interest in our premium coffee blend! We're thrilled to introduce our newest addition, carefully crafted to rival the best of the industry - Starbucks, Peets Coffee, and Cafe Coffee Day.

Our expert roasters have meticulously blended beans from around the world to create a truly unique flavor profile. Our signature blend is characterized


--- Run 3 ---
Generated Text:


Here's a warm and inviting response to the customer's inquiry:

"Thank you for reaching out about our new premium coffee blend! We're thrilled to introduce you to our latest creation, carefully crafted to rival the best of the industry. Our expert roasters have carefully selected the finest Arabica beans from around the world to create a truly unique flavor profile.

What sets our blend apart is its bold and


--- End Demonstration for this parameter setting ---


In [19]:
# Example 3.3: High top_p (for brainstorming diverse opening lines for marketing outreach)
generate_and_display(
    common_prompt_top_p,
    "top_p", 0.99, # Pushing higher for more noticeable effect
    num_runs=3,
    temperature=0.7,
    max_tokens=80
)


--- Demonstrating top_p=0.99 ---
Prompt: "
A customer is asking about our new premium coffee blend, that rivals Starbucks, Peets Coffee and Cafe Coffee Day. 
Provide a welcoming response that also highlights its unique flavor notes."
Other consistent parameters: {'temperature': 0.7, 'max_tokens': 80}

--- Run 1 ---
Generated Text:


"Welcome to [Your Business Name]! We're thrilled you're interested in our new premium coffee blend. I'd be happy to tell you more about it.

Our 'Cafe Royale' blend is indeed a game-changer, and many of our customers have already fallen in love with its rich, smooth flavor profile. What sets it apart from the big names like Starbucks, Peet's Coffee


--- Run 2 ---
Generated Text:


"Thank you for stopping by! I'd be delighted to tell you more about our new premium coffee blend. We're thrilled to introduce 'Elevé,' carefully crafted to rival the best in the industry - Starbucks, Peet's Coffee, and Cafe Coffee Day.

What sets our Elevé blend apart is its unique flavor profile, which combines rich notes of dark chocolate, hints of smoky caramel


--- Run 3 ---
Generated Text:


Welcome to our cafe! We're thrilled to introduce you to our newest premium coffee blend, carefully crafted to rival the best of the industry. Our expert roasters have worked tirelessly to create a truly unique and exceptional taste experience.

Our signature blend is a masterful fusion of South American and East African beans, showcasing rich notes of dark chocolate, hints of caramel, and subtle undertones of citrus. But


--- End Demonstration for this parameter setting ---


### 4. Frequency_penalty

**Commercial Use Cases**: Ensuring variety in marketing emails, preventing chatbots from repeating FAQs, generating diverse social media posts about the same product.

**What to Observe**:

- **No** penalty (0.0): Look for words or short phrases being repeated frequently within the generated paragraph.

- **Moderate** penalty: Repetition should be noticeably reduced, leading to more varied sentences.

- **High** penalty: The model will actively avoid repeating words, potentially leading to more complex or less natural phrasing if it has to find many synonyms.

In [20]:
common_prompt_freq_penalty = """
Give me 15 different phrases that customer support executives often say. Only return the phrases.
"""

In [21]:
# Example 4.1: Negative frequency penalty (encourages repetition)
generate_and_display(
    common_prompt_freq_penalty,
    "frequency_penalty", -1,
    temperature=0.7,
    seed = 42
)


--- Demonstrating frequency_penalty=-1 ---
Prompt: "
Give me 15 different phrases that customer support executives often say. Only return the phrases.
"
Other consistent parameters: {'temperature': 0.7, 'seed': 42}

--- Run 1 ---
Generated Text:


1. "I'd be happy to assist you further."
2. "Let me check on that for you."
3. "Can I offer you a solution?"
4. "I apologize for the inconvenience."
5. "How can I make it right for you?"
6. "I'd like to escalate this issue to my supervisor."
7. "Is there anything else I can help you with today?"
8. "I'm going to go ahead and process that for you."
9. "Can you please provide me with your order number?"
10. "I'll do my best to resolve the issue."
11. "I'd like to offer you a refund."
12. "I'm here to help you through this process."
13. "Can you confirm your email address?"
14. "I'll check on the status of that for you."
15. "Is there anything else I can do to assist you?"


--- End Demonstration for this parameter setting ---


In [22]:
# Example 4.2: Moderate frequency penalty (reduced repetition)
generate_and_display(
    common_prompt_freq_penalty,
    "frequency_penalty", 1.0, # Increased for more impact
    temperature=0.7,
    seed=42
)


--- Demonstrating frequency_penalty=1.0 ---
Prompt: "
Give me 15 different phrases that customer support executives often say. Only return the phrases.
"
Other consistent parameters: {'temperature': 0.7, 'seed': 42}

--- Run 1 ---
Generated Text:


1. "I'd be happy to assist you further."
2. "Let me check on that for you."
3. "Can you please confirm your order number?"
4. "Our policy is to provide a full refund in this situation."
5. "I've located your issue and we'll take care of it right away."
6. "You're being helped by John/Emily/David today, can I get his email?"
7. "Is there anything else I can help you with regarding this order?"
8. "Our current processing time is 24-48 hours for expedited orders."
9. "I'd like to offer an upgrade to our premium service plan."
10. "You've been selected for our loyalty rewards program, welcome!"
11. "Can you please try restarting your device and see if that resolves the issue?"
12. "Our estimated delivery date is now expected on [new date]."
13. "'Did you mean' to purchase [similar product] instead?"
14. "I can definitely escalate this issue for priority attention."
15. "You've been charged a fee of $25 due to excessive usage."


--- End Demonstration for this parameter setting ---


In [23]:
# Example 4.3: High frequency penalty (strong reduction in repetition, might affect coherence)
generate_and_display(
    common_prompt_freq_penalty,
    "frequency_penalty", 2.0, # Max penalty for strong effect
    temperature=0.7,
    seed=42
)


--- Demonstrating frequency_penalty=2.0 ---
Prompt: "
Give me 15 different phrases that customer support executives often say. Only return the phrases.
"
Other consistent parameters: {'temperature': 0.7, 'seed': 42}

--- Run 1 ---
Generated Text:


1. "I'd be happy to assist you further."
2. "Let's try again, shall we?"
3. "Can I provide an alternative solution?"
4."What would make this issue resolve for you?"
5."Have you tried [alternative steps]?" 
6.  No worries at all.
7."'re here to help now"
8.""I'm sorry about the inconvenience."
9.'"s a great question, let me see what I can do."
10.".Can I offer an upgrade/downgrade option?"
11."Okay, so just to confirm..."
12."Is there anything else you need assistance with today?"
13. "What's your preferred resolution for this issue?" 
14."'m going back into that system now"
15.""I'm here to help resolve the problem"


--- End Demonstration for this parameter setting ---


### 5. Presence_penalty

**Commercial Use Cases**: Encouraging a customer service bot to explore different solutions/topics, generating comprehensive meeting minutes by covering all points, producing diverse content ideas for a campaign.

**What to Observe**:

- **No** penalty (0.0): The model might dwell on a few initial topics or points without moving on to others mentioned in the conceptual prompt.

- **Moderate** penalty: The summary should cover a broader range of distinct action items or concepts, rather than just elaborating on the first few.

- **High** penalty: The model will try hard to introduce new concepts or distinct ideas, possibly jumping between points quickly or even generating somewhat disjointed output if forced too much.

In [24]:
common_prompt_pres_penalty = """
You are leading a quick internal meeting discussing how Generative AI can significantly improve our current sales and customer engagement processes. 
Briefly outline 4 distinct areas or strategies where GenAI could have a major impact, providing a brief example for each.
"""

In [25]:
# Example 5.1: No presence penalty (might focus heavily on one or two main themes)
generate_and_display(
    common_prompt_pres_penalty,
    "presence_penalty", 0.0,
    temperature=0.7,
)


--- Demonstrating presence_penalty=0.0 ---
Prompt: "
You are leading a quick internal meeting discussing how Generative AI can significantly improve our current sales and customer engagement processes. 
Briefly outline 4 distinct areas or strategies where GenAI could have a major impact, providing a brief example for each.
"
Other consistent parameters: {'temperature': 0.7}

--- Run 1 ---
Generated Text:


Let's dive into the potential of Generative AI to revolutionize our sales and customer engagement processes. Here are four distinct areas where we can expect significant improvements:

**1. Personalized Sales Content Generation**

GenAI can help us create personalized sales content, such as emails, social media posts, and sales scripts, that cater to individual customers' needs and preferences.

Example: We can use GenAI to generate a customized sales email for a lead who has shown interest in our product. The AI will analyze the customer's online behavior, previous interactions, and purchase history to craft a tailored message that resonates with their specific pain points and interests.

**2. Chatbot-Powered Customer Support**

GenAI-powered chatbots can help us provide faster, more efficient, and empathetic support to our customers, reducing response times and improving overall satisfaction.

Example: We can train GenAI to power a chatbot that responds to common customer inquiries about our product features, pricing, and availability. The AI will learn from user interactions to improve its responses over time, providing a seamless experience for customers who prefer self-service support.

**3. Customized Customer Segmentation**

GenAI can help us segment our customer base into distinct groups based on their behavior, preferences, and demographic characteristics, enabling us to tailor our marketing efforts and sales strategies more effectively.

Example: We can use GenAI to analyze customer data and identify patterns that reveal specific segments of our audience. For instance, the AI might detect a group of customers who are highly active on social media but have never made a purchase. By targeting this segment with targeted marketing campaigns and personalized content, we can increase engagement and conversion rates.

**4. Sales Forecasting and Predictive Analytics**

GenAI can help us predict sales performance by analyzing historical data, market trends, and customer behavior patterns, enabling us to make more informed forecasting decisions and optimize our sales strategies.

Example: We can train GenAI on historical sales data, market research reports, and industry trends to forecast future sales revenue. The AI will identify key drivers of growth and decline, allowing us to adjust our pricing, inventory levels, and sales channels accordingly, ultimately improving our bottom line and competitiveness.

These are just a few examples of the many ways in which Generative AI can enhance our sales and customer engagement processes. Do any of these areas resonate with you, or would you like to explore other applications?


--- End Demonstration for this parameter setting ---


In [26]:
# Example 5.2: Moderate presence penalty (encourages covering a broader range of distinct action items)
generate_and_display(
    common_prompt_pres_penalty,
    "presence_penalty", 1.0, # Increased for more impact
    temperature=0.7,
)


--- Demonstrating presence_penalty=1.0 ---
Prompt: "
You are leading a quick internal meeting discussing how Generative AI can significantly improve our current sales and customer engagement processes. 
Briefly outline 4 distinct areas or strategies where GenAI could have a major impact, providing a brief example for each.
"
Other consistent parameters: {'temperature': 0.7}

--- Run 1 ---
Generated Text:


Let's dive into the potential of Generative AI (GenAI) in improving our sales and customer engagement processes.

**Area 1: Personalized Content Generation**

* Strategy: Utilize GenAI to generate personalized content, such as blog posts, social media posts, and email campaigns, tailored to individual customers' interests and preferences.
* Example: Our marketing team uses a GenAI tool to analyze customer data and generates customized product recommendations for each user. For instance, if a customer browses our website frequently, the system suggests relevant products based on their browsing history.

**Area 2: Chatbot-Driven Customer Support**

* Strategy: Leverage GenAI-powered chatbots to provide 24/7 customer support, offering immediate assistance and resolving issues quickly.
* Example: Our company's genai-generated chatbot can respond to frequently asked questions, such as product prices and return policies. The bot also uses natural language processing (NLP) to understand the user's intent and direct them to human agents when needed.

**Area 3: Data-Driven Sales Analytics**

* Strategy: Harness GenAI's capabilities in data analysis to gain deeper insights into customer behavior and improve sales forecasting.
* Example: Our genai-powered analytics tool analyzes large datasets on customer purchasing patterns, seasonality, and market trends. This enables our sales team to identify high-potential opportunities and adjust their strategies accordingly.

**Area 4: Real-Time Customer Profiling**

* Strategy: Develop a GenAI-driven profiling system that creates accurate, up-to-date profiles of each customer based on their interactions with the company.
* Example: Our genai-powered customer profiling tool aggregates data from multiple sources (e.g., website sessions, social media engagement) to create rich, dynamic profiles. This information can help our sales team identify key decision-makers and tailor communications to specific personas.

These are just a few examples of how GenAI can enhance our sales and customer engagement processes. By exploring these areas, we can unlock significant opportunities for growth, efficiency, and improved customer experiences.


--- End Demonstration for this parameter setting ---


In [27]:
# Example 5.3: High presence penalty (strong encouragement for new ideas, potentially disjointed)
generate_and_display(
    common_prompt_pres_penalty,
    "presence_penalty", 2.0, # Max penalty for strong effect
    temperature=0.7,
)


--- Demonstrating presence_penalty=2.0 ---
Prompt: "
You are leading a quick internal meeting discussing how Generative AI can significantly improve our current sales and customer engagement processes. 
Briefly outline 4 distinct areas or strategies where GenAI could have a major impact, providing a brief example for each.
"
Other consistent parameters: {'temperature': 0.7}

--- Run 1 ---
Generated Text:


Here's an overview of four key areas where Generative AI (GenAI) can make significant improvements:

**1. Personalized Sales Content Generation**

* Strategy: Utilize GenAI to generate personalized sales content, such as email templates and social media posts, tailored to individual customers' needs and interests.
* Example: Our marketing team uses GenAI tools to create customized sales emails that address specific pain points of our target audience. For instance, if a customer has shown interest in purchasing software for project management, the AI-generated email will highlight how our tool can help streamline their workflow.

**2. Automated Customer Feedback Analysis**

* Strategy: Leverage GenAI-powered Natural Language Processing (NLP) to analyze and respond to customer feedback across various channels.
* Example: Our chatbots use GenAI algorithms to quickly scan through customer support emails, social media comments, and review sites to identify key themes and concerns. This enables us to address recurring issues proactively and improve overall customer satisfaction.

**3. Predictive Content Recommendation**

* Strategy: Integrate GenAI with our content management system (CMS) to predict which articles or blog posts will resonate most with specific audience segments.
* Example: Our marketing team uses GenAI tools to analyze historical data on article engagement, sentiment, and demographics to identify top-performing topics for each region. For instance, if we notice that users in the Midwest tend to engage more with content about local business trends, our AI-powered recommendation engine suggests creating similar pieces of content.

**4. Enhanced Customer Journey Mapping**

* Strategy: Apply GenAI's advanced visualization capabilities to create immersive and interactive customer journey maps.
* Example: Our sales team uses GenAI tools to generate 3D visualizations that illustrate the complexities of a potential client's buying process, highlighting where they might be most likely to encounter friction points. This allows us to tailor our approach and anticipate their needs more effectively.

By implementing these strategies leveraging Generative AI, we can significantly enhance our sales and customer engagement processes while providing better value for our customers.


--- End Demonstration for this parameter setting ---


### 6. Stop

**Commercial Use Cases**: Ensuring chatbot responses don't continue past a natural break, extracting specific data blocks (e.g., JSON), generating bulleted lists that don't overextend, completing fill-in-the-blank forms.
garding:"

**What to Observe**: The model's generation will immediately halt once any of the provided stop sequences appear in its output.

In [28]:
# Example 6.1: Stopping at newline (for single-line completions)
common_prompt_stop = """
Draft a quick customer service response template:\nThank you for contacting Reliance Jio. We received your inquiry regarding:"""
generate_and_display(
    common_prompt_stop,
    "stop", ["\n"],
    temperature=0.5
)


--- Demonstrating stop=['\n'] ---
Prompt: "
Draft a quick customer service response template:
Thank you for contacting Reliance Jio. We received your inquiry regarding:"
Other consistent parameters: {'temperature': 0.5}

--- Run 1 ---
Generated Text:


Here's a draft of a quick customer service response template:


--- End Demonstration for this parameter setting ---


In [29]:
# Example 6.2: Stopping at a specific closing phrase (for structured email generation)
common_prompt_email = """Draft a professional email to a client regarding their order status. 
Subject: Your Order #12345 Update\nDear [Client Name],\n\nYour recent order #12345 has been:"""
generate_and_display(
    common_prompt_email,
    "stop", ["Best regards,"], # Model will stop before adding the closing
    temperature=0.5
)


--- Demonstrating stop=['Best regards,'] ---
Prompt: "Draft a professional email to a client regarding their order status. 
Subject: Your Order #12345 Update
Dear [Client Name],

Your recent order #12345 has been:"
Other consistent parameters: {'temperature': 0.5}

--- Run 1 ---
Generated Text:


Here's a draft of the email:

Subject: Your Order #12345 Update

Dear [Client Name],

We are pleased to inform you that your recent order, #12345, is currently being processed and dispatched. Our team has confirmed that all necessary steps have been taken to prepare your shipment for delivery.

As per our previous communication, we anticipate that your order will arrive at your doorstep within the estimated timeframe of [insert timeframe]. You can track the status of your order using the tracking number: [insert tracking number].

If you have any questions or concerns regarding your order, please do not hesitate to reach out to us. We are here to assist you and ensure a seamless experience.

Thank you for your patience and understanding.


--- End Demonstration for this parameter setting ---


In [30]:
# Example 6.3: Stopping after a list item (for controlled list generation)
common_prompt_list = "List 3 benefits of our premium AI/ML package:\n1. Enhanced security\n2. Real-time analytics\n3."
generate_and_display(
    common_prompt_list,
    "stop", ["4."], # Stop before generating the 4th item (assuming it would start with 4.)
    temperature=0.5
)


--- Demonstrating stop=['4.'] ---
Prompt: "List 3 benefits of our premium AI/ML package:
1. Enhanced security
2. Real-time analytics
3."
Other consistent parameters: {'temperature': 0.5}

--- Run 1 ---
Generated Text:


Here is the completed list:

**Benefits of Our Premium AI/ML Package:**

1. **Enhanced Security**: Our advanced AI-powered threat detection and prevention capabilities protect your business from sophisticated cyber threats, ensuring sensitive data remains secure.
2. **Real-time Analytics**: Get instant insights into customer behavior, market trends, and operational performance with our cutting-edge analytics platform, empowering data-driven decision-making.
3. **Predictive Modeling and Automation**: Leverage our machine learning algorithms to predict customer churn, optimize resource allocation, and automate repetitive tasks, freeing up your team to focus on strategic initiatives.


--- End Demonstration for this parameter setting ---


### 7. Seed

**Commercial Use Cases**: Reproducible content for A/B testing (ensuring differences are due to prompt, not randomness), consistent chatbot behavior for quality assurance, debugging prompt engineering, generating consistent test data.

**What to Observe**:

- **Same** seed: The outputs for the multiple runs should be identical or extremely close.
- **Different** seed: The outputs, while still relevant to the prompt, should be distinct from each other across different seeds.

In [31]:
common_prompt_seed = "Generate a marketing slogan for a new eco-friendly cleaning product. Only return the slogan."

In [32]:
# Example 7.1: Same seed, same prompt, same parameters (for reproducible slogan generation for testing)
print("\n--- Testing reproducibility with the SAME seed (42) ---")
generate_and_display(
    common_prompt_seed,
    "seed", 42,
    num_runs=3, # Run multiple times to show consistency
    temperature=0.7,
    max_tokens=20
)


--- Testing reproducibility with the SAME seed (42) ---

--- Demonstrating seed=42 ---
Prompt: "Generate a marketing slogan for a new eco-friendly cleaning product. Only return the slogan."
Other consistent parameters: {'temperature': 0.7, 'max_tokens': 20}

--- Run 1 ---
Generated Text:


"Clean with a Clear Conscience, Naturally."


--- Run 2 ---
Generated Text:


"Clean with a Clear Conscience, Naturally."


--- Run 3 ---
Generated Text:


"Clean with a Clear Conscience, Naturally."


--- End Demonstration for this parameter setting ---


In [33]:
# Example 7.2: Different seeds, same prompt, same parameters (to get diverse options for review)
print("\n--- Testing different seeds for DIVERSE slogan options ---")
generate_and_display(
    common_prompt_seed,
    "seed", 101,
    num_runs=1, # One run for each distinct seed is enough to show variety
    temperature=0.7,
    max_tokens=20
)


--- Testing different seeds for DIVERSE slogan options ---

--- Demonstrating seed=101 ---
Prompt: "Generate a marketing slogan for a new eco-friendly cleaning product. Only return the slogan."
Other consistent parameters: {'temperature': 0.7, 'max_tokens': 20}

--- Run 1 ---
Generated Text:


"Sparkle with a Clear Conscience, Clean with Green Intent."


--- End Demonstration for this parameter setting ---


In [34]:
# Example 7.3: Different seeds, same prompt, same parameters (to get diverse options for review)
generate_and_display(
    common_prompt_seed,
    "seed", 202,
    num_runs=1,
    temperature=0.7,
    max_tokens=20
)


--- Demonstrating seed=202 ---
Prompt: "Generate a marketing slogan for a new eco-friendly cleaning product. Only return the slogan."
Other consistent parameters: {'temperature': 0.7, 'max_tokens': 20}

--- Run 1 ---
Generated Text:


"Clean with a Clear Conscience, Inside and Out"


--- End Demonstration for this parameter setting ---


In [35]:
# Example 7.4: Different seeds, same prompt, same parameters (to get diverse options for review)
generate_and_display(
    common_prompt_seed,
    "seed", 303,
    num_runs=1,
    temperature=0.7,
    max_tokens=20
)


--- Demonstrating seed=303 ---
Prompt: "Generate a marketing slogan for a new eco-friendly cleaning product. Only return the slogan."
Other consistent parameters: {'temperature': 0.7, 'max_tokens': 20}

--- Run 1 ---
Generated Text:


"Cleaning your home, greening our planet."


--- End Demonstration for this parameter setting ---
